# Import

In [10]:
# #.py modules
# import acquire as ac
import prepare as p
# import explore as ex
# import evaluate as ev
import wrangle as w

#standard
import pandas as pd
import numpy as np
import re

#scraping
import requests
from requests import get
from bs4 import BeautifulSoup

#file
import os
import json
from google.oauth2 import service_account
import pandas_gbq

#vizz
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud

#preprocess
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

#split and model
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

#set random state
random_state=123

#ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [11]:
# credentials = service_account.Credentials.from_service_account_file(
#     'my-ds-projects-d864a770b51b.json',
# )

In [12]:
# df = pandas_gbq.read_gbq("select * from cfpb_complaints.complaint_database",dialect='standard',project_id="my-ds-projects",use_bqstorage_api=True,credentials=credentials)
# df

Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


,date_received,product,subproduct,issue,subissue,consumer_complaint_narrative,company_public_response,company_name,state,zip_code,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed,complaint_id
0,2016-05-19,Money transfers,Domestic (US) money transfer,Wrong amount charged or received,None,western union repeatedly charged me XXXX per t...,None,"WESTERN UNION COMPANY, THE",PA,18705,None,Consent provided,Web,2016-05-19,Closed with explanation,True,False,1933427
1,2022-08-01,"Payday loan, title loan, or personal loan",Payday loan,"Was approved for a loan, but didn't receive th...",None,None,None,OneMain Finance Corporation,MD,21207.0,None,Consent not provided,Web,2022-08-01,Closed with explanation,True,<NA>,5832898
2,2016-12-01,Debt collection,I do not know,Taking/threatening an illegal action,Seized/Attempted to seize property,None,Company believes it acted appropriately as aut...,"Credit Central Holdings, LLC",AL,35007,None,Consent not provided,Web,2016-12-01,Closed with explanation,True,False,2231515
3,2018-12-27,Vehicle loan or lease,Loan,Problems at the end of the loan or lease,Problem while selling or giving up the vehicle,None,None,PRESTIGE FINANCIAL SERVICES INC,WA,98032.0,None,None,Referral,2018-12-27,Closed with explanation,True,<NA>,3111102
4,2016-12-20,Payday loan,None,Charged fees or interest I didn't expect,None,None,Company disputes the facts presented in the co...,"ENOVA INTERNATIONAL, INC.",VA,24018,None,None,Referral,2016-12-20,Closed with explanation,True,False,2258627
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3458901,2012-10-08,Mortgage,Conventional fixed mortgage,"Loan modification,collection,foreclosure",None,None,None,WELLS FARGO & COMPANY,FL,33319,None,None,Web,2012-10-09,Closed with explanation,True,True,167180
3458902,2013-12-06,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",None,None,None,WELLS FARGO & COMPANY,GA,30032,Servicemember,None,Fax,2013-12-09,Closed with explanation,True,False,626697
3458903,2018-05-20,Mortgage,Conventional home mortgage,Trouble during payment process,None,None,Company has responded to the consumer and the ...,WELLS FARGO & COMPANY,NV,89801.0,Older American,Consent not provided,Web,2018-05-20,Closed with explanation,True,<NA>,2912388
3458904,2019-06-03,Mortgage,Conventional home mortgage,Trouble during payment process,None,None,Company has responded to the consumer and the ...,WELLS FARGO & COMPANY,CA,92336.0,None,None,Referral,2019-06-04,Closed with explanation,True,<NA>,3264276


In [13]:
# df.to_csv('ccr.csv')

In [21]:
df = pd.read_csv('ccr.csv')

In [22]:
df.head()

,Unnamed: 0,date_received,product,subproduct,issue,subissue,consumer_complaint_narrative,company_public_response,company_name,state,zip_code,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed,complaint_id
0,0,2016-05-19,Money transfers,Domestic (US) money transfer,Wrong amount charged or received,NaN,western union repeatedly charged me XXXX per t...,NaN,"WESTERN UNION COMPANY, THE",PA,18705.0,NaN,Consent provided,Web,2016-05-19,Closed with explanation,True,False,1933427
1,1,2022-08-01,"Payday loan, title loan, or personal loan",Payday loan,"Was approved for a loan, but didn't receive th...",NaN,NaN,NaN,OneMain Finance Corporation,MD,21207.0,NaN,Consent not provided,Web,2022-08-01,Closed with explanation,True,NaN,5832898
2,2,2016-12-01,Debt collection,I do not know,Taking/threatening an illegal action,Seized/Attempted to seize property,NaN,Company believes it acted appropriately as aut...,"Credit Central Holdings, LLC",AL,35007.0,NaN,Consent not provided,Web,2016-12-01,Closed with explanation,True,False,2231515
3,3,2018-12-27,Vehicle loan or lease,Loan,Problems at the end of the loan or lease,Problem while selling or giving up the vehicle,NaN,NaN,PRESTIGE FINANCIAL SERVICES INC,WA,98032.0,NaN,NaN,Referral,2018-12-27,Closed with explanation,True,NaN,3111102
4,4,2016-12-20,Payday loan,NaN,Charged fees or interest I didn't expect,NaN,NaN,Company disputes the facts presented in the co...,"ENOVA INTERNATIONAL, INC.",VA,24018.0,NaN,NaN,Referral,2016-12-20,Closed with explanation,True,False,2258627


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3458906 entries, 0 to 3458905
Data columns (total 19 columns):
 #   Column                        Dtype 
---  ------                        ----- 
 0   Unnamed: 0                    int64 
 1   date_received                 object
 2   product                       object
 3   subproduct                    object
 4   issue                         object
 5   subissue                      object
 6   consumer_complaint_narrative  object
 7   company_public_response       object
 8   company_name                  object
 9   state                         object
 10  zip_code                      object
 11  tags                          object
 12  consumer_consent_provided     object
 13  submitted_via                 object
 14  date_sent_to_company          object
 15  company_response_to_consumer  object
 16  timely_response               bool  
 17  consumer_disputed             object
 18  complaint_id                  int64 
dtype

In [24]:
df.describe()

,Unnamed: 0,complaint_id
count,3.458906e+06,3.458906e+06
mean,1.729452e+06,3.831191e+06
std,9.985003e+05,1.790616e+06
min,0.000000e+00,1.000000e+00
25%,8.647262e+05,2.669770e+06
50%,1.729452e+06,3.805222e+06
75%,2.594179e+06,5.415978e+06
max,3.458905e+06,6.741992e+06


In [25]:
w.data_summary(df)

data shape: (3458906, 19)


,data type,#missing,%missing,#unique,count,mean,std,min,25%,50%,75%,max,first_value,second_value,third_value
unnamed: 0,int64,0,0.000000,3458906,3458906.0,1729452.5,998500.299439,0.0,864726.25,1729452.5,2594178.75,3458905.0,0,1,2
date_received,object,0,0.000000,4129,3458906,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-05-19,2022-08-01,2016-12-01
product,object,0,0.000000,18,3458906,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Money transfers,"Payday loan, title loan, or personal loan",Debt collection
subproduct,object,235291,6.802469,76,3223615,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Domestic (US) money transfer,Payday loan,I do not know
issue,object,0,0.000000,165,3458906,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wrong amount charged or received,"Was approved for a loan, but didn't receive th...",Taking/threatening an illegal action
subissue,object,699902,20.234779,221,2759004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Seized/Attempted to seize property
consumer_complaint_narrative,object,2212167,63.955684,1080431,1246739,NaN,NaN,NaN,NaN,NaN,NaN,NaN,western union repeatedly charged me XXXX per t...,NaN,NaN
company_public_response,object,1921950,55.565257,11,1536956,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Company believes it acted appropriately as aut...
company_name,object,0,0.000000,6694,3458906,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"WESTERN UNION COMPANY, THE",OneMain Finance Corporation,"Credit Central Holdings, LLC"
state,object,41114,1.188642,63,3417792,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PA,MD,AL


## data response is the target

In [14]:
p.basic_clean(df)

AttributeError: 'DataFrame' object has no attribute 'lower'

In [29]:
import os
from google.cloud import bigquery

# Set the environment variable
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'my-ds-projects-d864a770b51b.json'

# Initialize a BigQuery client
client = bigquery.Client()

# Now you can use the client...


In [43]:
from google.cloud import bigquery

# Initialize a BigQuery client
client = bigquery.Client()

# Define your query
query = """
    SELECT *
    FROM `bigquery-public-data.cfpb_complaints.complaint_database`
"""

# Run the query
query_job = client.query(query)

# Get the results as a pandas DataFrame
df_2 = query_job.to_dataframe()

# Now you can analyze the data using pandas, for example:
summary = df_2.describe(include='all')


In [44]:
summary

,date_received,product,subproduct,issue,subissue,consumer_complaint_narrative,company_public_response,company_name,state,zip_code,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed,complaint_id
count,3458906,3458906,3223615,3458906,2759004,1246739,1536956,3458906,3417792,3417274,382213,2588777,3458906,3458906,3458902,3458906,768399,3458906
unique,4129,18,76,165,221,1080431,11,6694,63,60303,3,4,7,4078,8,2,2,3458906
top,2023-01-20,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,My credit reports are inaccurate. These inaccu...,Company has responded to the consumer and the ...,"EQUIFAX, INC.",CA,30349.0,Servicemember,Consent provided,Web,2023-02-28,Closed with explanation,True,False,5777486
freq,4494,1714173,1695867,858157,553249,1750,1321028,619370,414208,4589,216614,1247422,2950186,4521,2612184,3406950,620021,1


In [45]:
df_2.to_csv('cfpb.csv')

In [46]:
# Print the first few rows of the DataFrame
df_2.head()

,date_received,product,subproduct,issue,subissue,consumer_complaint_narrative,company_public_response,company_name,state,zip_code,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed,complaint_id
0,2022-07-16,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,None,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,PA,19136,None,Consent not provided,Web,2022-07-16,Closed with explanation,True,<NA>,5777486
1,2023-01-23,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,None,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,NV,89139.0,None,Consent not provided,Web,2023-01-23,Closed with explanation,True,<NA>,6476526
2,2022-06-15,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,None,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,NC,27591.0,None,Consent not provided,Web,2022-06-15,Closed with explanation,True,<NA>,5675690
3,2022-10-09,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,None,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,OH,44266.0,None,Consent not provided,Web,2022-10-09,Closed with non-monetary relief,True,<NA>,6064894
4,2019-03-02,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information is missing that should be on the r...,"I was locked out of my credit report by XXXX, ...",Company has responded to the consumer and the ...,Experian Information Solutions Inc.,IL,62702.0,None,Consent provided,Web,2019-03-02,Closed with explanation,True,<NA>,3167321


In [48]:
# Print the summary statistics of the DataFrame
print(df_2.describe())

       date_received                                            product  \
count        3458906                                            3458906   
unique          4129                                                 18   
top       2023-01-20  Credit reporting, credit repair services, or o...   
freq            4494                                            1714173   

              subproduct                                 issue  \
count            3223615                               3458906   
unique                76                                   165   
top     Credit reporting  Incorrect information on your report   
freq             1695867                                858157   

                                   subissue  \
count                               2759004   
unique                                  221   
top     Information belongs to someone else   
freq                                 553249   

                             consumer_complaint_narrative

In [49]:
# Print the column names
print(df_2.columns)

Index(['date_received', 'product', 'subproduct', 'issue', 'subissue',
       'consumer_complaint_narrative', 'company_public_response',
       'company_name', 'state', 'zip_code', 'tags',
       'consumer_consent_provided', 'submitted_via', 'date_sent_to_company',
       'company_response_to_consumer', 'timely_response', 'consumer_disputed',
       'complaint_id'],
      dtype='object')


In [50]:
# Count the number of non-NA values
print(df_2.count())

date_received                   3458906
product                         3458906
subproduct                      3223615
issue                           3458906
subissue                        2759004
consumer_complaint_narrative    1246739
company_public_response         1536956
company_name                    3458906
state                           3417792
zip_code                        3417274
tags                             382213
consumer_consent_provided       2588777
submitted_via                   3458906
date_sent_to_company            3458906
company_response_to_consumer    3458902
timely_response                 3458906
consumer_disputed                768399
complaint_id                    3458906
dtype: int64


In [51]:
# Compute pairwise correlation of columns
print(df_2.corr())

                   timely_response  consumer_disputed
timely_response           1.000000           0.031421
consumer_disputed         0.031421           1.000000


In [ ]:
df.dropna()

In [53]:
import nltk

nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/rosendo/nltk_data...


True

In [56]:
from nltk.sentiment import SentimentIntensityAnalyzer

# Initialize the sentiment intensity analyzer
sia = SentimentIntensityAnalyzer()

# Replace None with '' (empty string)
df_2['consumer_complaint_narrative'] = df_2['consumer_complaint_narrative'].fillna('')

# Apply the sentiment intensity analyzer to the 'consumer_complaint_narrative' column
df_2['sentiment'] = df_2['consumer_complaint_narrative'].apply(lambda complaint: sia.polarity_scores(complaint))

# Now each complaint has a sentiment score


In [57]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3458906 entries, 0 to 3458905
Data columns (total 19 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   date_received                 dbdate 
 1   product                       object 
 2   subproduct                    object 
 3   issue                         object 
 4   subissue                      object 
 5   consumer_complaint_narrative  object 
 6   company_public_response       object 
 7   company_name                  object 
 8   state                         object 
 9   zip_code                      object 
 10  tags                          object 
 11  consumer_consent_provided     object 
 12  submitted_via                 object 
 13  date_sent_to_company          dbdate 
 14  company_response_to_consumer  object 
 15  timely_response               boolean
 16  consumer_disputed             boolean
 17  complaint_id                  object 
 18  sentiment             

In [59]:
df_2.product()

timely_response      0
consumer_disputed    0
dtype: int64

In [61]:
df_2.head()

,date_received,product,subproduct,issue,subissue,consumer_complaint_narrative,company_public_response,company_name,state,zip_code,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed,complaint_id,sentiment
0,2022-07-16,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,PA,19136,None,Consent not provided,Web,2022-07-16,Closed with explanation,True,<NA>,5777486,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound..."
1,2023-01-23,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,NV,89139.0,None,Consent not provided,Web,2023-01-23,Closed with explanation,True,<NA>,6476526,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound..."
2,2022-06-15,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,NC,27591.0,None,Consent not provided,Web,2022-06-15,Closed with explanation,True,<NA>,5675690,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound..."
3,2022-10-09,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,OH,44266.0,None,Consent not provided,Web,2022-10-09,Closed with non-monetary relief,True,<NA>,6064894,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound..."
4,2019-03-02,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information is missing that should be on the r...,"I was locked out of my credit report by XXXX, ...",Company has responded to the consumer and the ...,Experian Information Solutions Inc.,IL,62702.0,None,Consent provided,Web,2019-03-02,Closed with explanation,True,<NA>,3167321,"{'neg': 0.095, 'neu': 0.813, 'pos': 0.092, 'co..."


In [64]:
df_2['product'].value_counts()

Credit reporting, credit repair services, or other personal consumer reports    1714173
Debt collection                                                                  473071
Mortgage                                                                         373639
Credit card or prepaid card                                                      180982
Checking or savings account                                                      158988
Credit reporting                                                                 140429
Credit card                                                                       89190
Bank account or service                                                           86205
Student loan                                                                      73648
Money transfer, virtual currency, or money service                                52280
Vehicle loan or lease                                                             41625
Consumer Loan                   

In [65]:
df_2['subproduct'].value_counts()

Credit reporting                              1695867
Checking account                               185602
General-purpose credit card or charge card     137889
I do not know                                  104301
Other debt                                      90462
                                               ...   
Credit repair                                     102
Traveler’s/Cashier’s checks                        88
Transit card                                       37
Student prepaid card                               29
Electronic Benefit Transfer / EBT card             12
Name: subproduct, Length: 76, dtype: int64

In [66]:
df_2['issue'].value_counts()

Incorrect information on your report                                                858157
Problem with a credit reporting company's investigation into an existing problem    456973
Improper use of your report                                                         372875
Attempts to collect debt not owed                                                   164028
Loan modification,collection,foreclosure                                            112307
                                                                                     ...  
Property was sold                                                                        9
Lender damaged or destroyed vehicle                                                      8
Lender sold the property                                                                 7
Property was damaged or destroyed property                                               6
Lender damaged or destroyed property                                                     3

In [67]:
df_2['company_name'].value_counts()

EQUIFAX, INC.                             619370
TRANSUNION INTERMEDIATE HOLDINGS, INC.    526785
Experian Information Solutions Inc.       492153
BANK OF AMERICA, NATIONAL ASSOCIATION     124251
WELLS FARGO & COMPANY                     111736
                                           ...  
Huston & Angus                                 1
Thomason Law Firm, LLC                         1
Argolica, LLC                                  1
Mercadolar, Inc.                               1
Crawford & von Keller, LLC                     1
Name: company_name, Length: 6694, dtype: int64

In [68]:
df_2['state'].value_counts()

CA    414208
FL    389321
TX    341294
NY    224142
GA    224063
       ...  
AA        53
MP        51
AS        38
MH        33
PW        13
Name: state, Length: 63, dtype: int64

In [70]:
df_2['zip_code'].value_counts()

30349.0    4589
35405.0    4144
19143.0    3672
33025.0    3496
77449.0    3242
           ... 
15846         1
50228.0       1
47868.0       1
45766.0       1
10932         1
Name: zip_code, Length: 60303, dtype: int64

In [71]:
df_2['tags'].value_counts()

Servicemember                    216614
Older American                   133725
Older American, Servicemember     31874
Name: tags, dtype: int64

In [72]:
df_2['consumer_consent_provided'].value_counts()

Consent provided        1247422
Consent not provided    1238109
Other                     97165
Consent withdrawn          6081
Name: consumer_consent_provided, dtype: int64

In [73]:
df_2['submitted_via'].value_counts()

Web             2950186
Referral         239888
Phone            154119
Postal mail       87903
Fax               25659
Web Referral        726
Email               425
Name: submitted_via, dtype: int64

In [74]:
df_2['company_response_to_consumer'].value_counts()

Closed with explanation            2612184
Closed with non-monetary relief     563627
Closed with monetary relief         129259
In progress                         103560
Closed without relief                17868
Closed                               17611
Untimely response                     9489
Closed with relief                    5304
Name: company_response_to_consumer, dtype: int64

In [75]:
df_2['timely_response'].value_counts()

True     3406950
False      51956
Name: timely_response, dtype: Int64

In [76]:
df_2['consumer_disputed'].value_counts()

False    620021
True     148378
Name: consumer_disputed, dtype: Int64

In [78]:
df_2['sentiment']

0          {'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound...
1          {'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound...
2          {'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound...
3          {'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound...
4          {'neg': 0.095, 'neu': 0.813, 'pos': 0.092, 'co...
                                 ...                        
3458901    {'neg': 0.0, 'neu': 0.961, 'pos': 0.039, 'comp...
3458902    {'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound...
3458903    {'neg': 0.187, 'neu': 0.662, 'pos': 0.151, 'co...
3458904    {'neg': 0.068, 'neu': 0.782, 'pos': 0.15, 'com...
3458905    {'neg': 0.142, 'neu': 0.798, 'pos': 0.06, 'com...
Name: sentiment, Length: 3458906, dtype: object